In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
#chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 검색어, 시작 날짜, 종료 날짜 입력
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"

seen_urls = set() 

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    base_url = "https://www.busan.com/search/index.php?search_string="
    results = []
    page = 1

    while True:
        print(f"{keyword} {page}")
        search_url = f"{base_url}{urllib.parse.quote(keyword)}&page={page}&cnt={(page - 1) * 10}&sort=DATE/DESC&start_date={start_date}&end_date={end_date}&real_search={urllib.parse.quote(keyword)}&rescan_chk=0&type=B&search_writer=undefined"
        driver.get(search_url)
        
        try:
            # 페이지 로드 완료 대기
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "search_list"))
            )
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 기사 목록 가져오기
            articles_section = soup.find("ul", id="search_list")

            articles = articles_section.find_all("li") if articles_section else []
            if not articles:
                print(f"더 이상 기사가 없습니다. 페이지 {page}에서 종료합니다.")
                break

            # 기사 정보 추출
            for item in articles:
                try:
                    title_tag = item.find("p", class_="title").find("a")
                    if title_tag:
                        title = title_tag.text.strip()
                        url = "https://www.busan.com" + title_tag["href"]
                    else:
                        title = ""
                        url = ""

                    # 중복 URL 체크
                    if url in seen_urls:
                        continue  # 이미 처리된 URL은 무시

                    seen_urls.add(url)  # 새로운 URL 저장
                    
                    date_element = item.find("p", class_="date")
                    date = date_element.text.strip().split()[0] if date_element else ""
                    # 결과 리스트에 추가
                    results.append({
                        'media': '부산일보', 
                        'title': title, 
                        'date': date, 
                        'section': '', 
                        'article_type': '', 
                        'writer': '', 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
                except Exception as e:
                    print(f"Error processing article: {e}")

        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break
        
        page += 1  # 다음 페이지로 이동
    
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

all_results = []
for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# Save results to Excel
save_to_excel(all_results, f'2부산일보 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        article_body = soup.find('div', class_='article_content')

        # 서브 타이틀이 있으면 추출하고 제거
        subtitle_tag = soup.find('p', class_='subtitle')
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 서브 타이틀 제거
        else:
            subtitle = ""

        if article_body:
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('div', class_='img_box img_center'):
                caption.decompose()
            tag = article_body.find('div', class_='h20')
            if tag:
                next_sibling = tag.find_next_sibling()
                while next_sibling:
                    next_sibling.decompose()
                    next_sibling = tag.find_next_sibling()
                tag.decompose()  # btn_area 자체도 제거 
            
            content = article_body.get_text(separator='\n', strip=True)
            
        else:
            content = ""
        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None
    
# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []  # 에러 로그 리스트 초기화
total_articles = len(df)  # 전체 기사 수 계산    

# 각 행의 URL에서 본문 내용을 스크랩하여 content와 subtitle 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open('error_log_path', 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")      
